In [1]:
import pandas as pd
import datetime
import json

from catboost import CatBoostClassifier, Pool
import numpy as np

In [2]:
import psycopg2

conn = psycopg2.connect(
        user="ivanovo",
        password="ivanovo",
        host="92.51.39.188",  # локальный хост
        port="5432",       # стандартный порт PostgreSQL
        database="ivanovo"
    )

In [30]:
model = CatBoostClassifier(
    iterations=1000,         # Количество итераций (увеличьте при необходимости)
    # learning_rate=0.1,       # Скорость обучения
    depth=7,                 # Глубина деревьев
    loss_function='MultiClass', # Функция потерь
    cat_features=categorical_features,
    random_seed=42,
    verbose=100)
model.load_model('model_task1')
model

In [27]:
features = [
 'УНОМ',
 'month',
 'mean_volume1forhour',
 'std_volume1forhour',
 'mean_volume2forhour',
 'std_volume2forhour',
 'mean_q2forhour',
 'std_q2forhour',
 'min_volume1forhour',
 'max_volume1forhour',
 'median_volume1forhour',
 'min_volume2forhour',
 'max_volume2forhour',
 'median_volume2forhour',
 'min_q2forhour',
 'max_q2forhour',
 'median_q2forhour',
 'Потребители',
 'Группа',
 'Центральное отопление(контур)',
 'Ошибки',
 'weather1',
 'weather2',
 'Округ',
 'Район',
 'Серии проектов',
 'Количество этажей',
 'Количество подъездов',
 'Количество квартир',
 'Общая площадь',
 'Общая площадь жилых помещений',
 'Износ объекта (по БТИ)',
 'Материалы стен',
 'Признак аварийности здания',
 'Количество пассажирских лифтов',
 'Количество грузопассажирских лифтов',
 'Материалы кровли по БТИ',
 'Типы жилищного фонда',
 'Статусы МКД',
 'Марка счетчика ',
 'ЦТП',
 'Муниципальный округ',
 'Материал',
 'Назначение',
 'Класс',
 'Тип',
 'OBJ_TYPE',
 'Внутригородская территория',
 'Давление не в норме_count',
 'T < min_count',
 'T > max_count',
 'Утечка_count',
 'month1_count',
 'month2_count',
 'month3_count',
 'month4_count',
 'month10_count',
 'month11_count',
 'month12_count',
 'year',
 'day',
 'hour',
 'dayofweek',
 'weekofyear',
 'quarter',
 'is_month_start',
 'is_month_end',
 'month_sin',
 'month_cos',
 'hour_sin',
 'hour_cos',
 'quarter_sin',
 'quarter_cos',
 'day_sin',
 'day_cos',
 'dayofweek_sin',
 'dayofweek_cos',
 'weekofyear_sin',
 'weekofyear_cos'
]

In [28]:
features2aggdata = [
    "УНОМ",
    'Округ',
 'Район',
 'Серии проектов',
 'Количество этажей',
 'Количество подъездов',
 'Количество квартир',
 'Общая площадь',
 'Общая площадь жилых помещений',
 'Износ объекта (по БТИ)',
 'Материалы стен',
 'Признак аварийности здания',
 'Количество пассажирских лифтов',
 'Количество грузопассажирских лифтов',
 'Материалы кровли по БТИ',
 'Типы жилищного фонда',
 'Статусы МКД',
 'Марка счетчика ',
 'ЦТП',
 'Муниципальный округ',
 'Материал',
 'Назначение',
 'Класс',
 'Тип',
 'OBJ_TYPE',
 'Внутригородская территория',

]


In [29]:
categorical_features = [
 'УНОМ',
 'month',
 'Потребители',
 'Группа',
 'Центральное отопление(контур)',
 'Ошибки',
 'Округ',
 'Район',
 'Серии проектов',
 'Материалы стен',
 'Признак аварийности здания',
 'Материалы кровли по БТИ',
 'Типы жилищного фонда',
 'Статусы МКД',
 'Марка счетчика ',
 'ЦТП',
 'Муниципальный округ',
 'Материал',
 'Назначение',
 'Класс',
 'Тип',
 'OBJ_TYPE',
 'Внутригородская территория',
 'is_month_start',
'is_month_end'
           ]

In [ ]:
%%time

def get_predict_for_all(model: CatBoostClassifier, date: datetime.datetime) -> dict: 

    conn = psycopg2.connect(
        user="ivanovo",
        password="ivanovo",
        host="92.51.39.188",  # локальный хост
        port="5432",       # стандартный порт PostgreSQL
        database="ivanovo"
    )
    
    agg_data = get_agg_data(conn)
    agg_data = agg_data.dropna(subset='unom')
    unomlst = agg_data['unom'].unique().tolist()
    n_preds = len(unomlst)
    events2preds = pd.DataFrame({
        "УНОМ": unomlst,
        "Дата создания во внешней системе": [date] * n_preds,
        "month": [date.month] * n_preds,
        "day": [date.day] * n_preds
    })
    
    odpu = get_odpu(date, conn)
    events2preds[ftrs2odpu] = events2preds.apply(lambda x: add_opdu_features(odpu, x), axis=1)

    with open('weather.json', 'r', encoding='utf-8') as file:
        weather = json.load(file)
    events2preds[['weather1', 'weather2']] = events2preds.apply(lambda x: collect_weather(x, weather), axis=1)

    events = get_events(date, conn)

    conn.close()
    events2preds[feature2events] = events2preds.apply(lambda x: collect_events(x, events), axis=1)
    agg_data['УНОМ'] = agg_data['unom']
    events2preds = events2preds.merge(agg_data[features2aggdata], how='left', on='УНОМ')

    
    events2preds = extract_datetime_features(events2preds, "Дата создания во внешней системе")
    events2preds = add_cyclic_features(events2preds, 'month', 12)
    events2preds = add_cyclic_features(events2preds, 'hour', 24)
    events2preds = add_cyclic_features(events2preds, 'quarter', 4)
    events2preds = add_cyclic_features(events2preds, 'day', 31)
    events2preds = add_cyclic_features(events2preds, 'dayofweek', 31)
    events2preds = add_cyclic_features(events2preds, 'weekofyear', 31)
    events2preds.drop("Дата создания во внешней системе", axis=1, inplace=True)
    

    numerical_features = [col for col in events2preds.columns if col not in categorical_features]
    
    # Удаление строк с NaN в числовых признаках
    events2preds[numerical_features] = events2preds[numerical_features].fillna(events2preds[numerical_features].mean())
    
    
    
    for col in categorical_features:
        events2preds[col] = events2preds[col].astype(str)
        
    preds = model.predict_proba(events2preds[features])
    events2preds['УНОМ'] = events2preds['УНОМ'].astype(float).astype(int)
    events2preds['preds'] = preds.tolist()
    
    return events2preds[['УНОМ', 'preds']].set_index('УНОМ')['preds'].to_dict()

result_dict = get_predict_for_all(model, datetime.datetime(2023, 12, 1))
result_dict

In [44]:
def get_predict_for_one(model: CatBoostClassifier, unom: int, date: datetime.datetime) -> dict:
    conn = psycopg2.connect(
        user="ivanovo",
        password="ivanovo",
        host="92.51.39.188",  # локальный хост
        port="5432",       # стандартный порт PostgreSQL
        database="ivanovo"
    )
    
    agg_data = get_agg_data_one(unom, conn)
    events2preds = pd.DataFrame({
        "УНОМ": [unom],
        "Дата создания во внешней системе": [date],
        "month": [date.month],
        "day": [date.day]
    })

    odpu = get_odpu_one(unom, date, conn)
    events2preds[ftrs2odpu] = events2preds.apply(lambda x: add_opdu_features(odpu, x), axis=1)

    with open('weather.json', 'r', encoding='utf-8') as file:
        weather = json.load(file)
    events2preds[['weather1', 'weather2']] = events2preds.apply(lambda x: collect_weather(x, weather), axis=1)

    events = get_events_one(unom, date, conn)

    conn.close()
    events2preds[feature2events] = events2preds.apply(lambda x: collect_events(x, events), axis=1)
    agg_data['УНОМ'] = agg_data['unom']
    events2preds = events2preds.merge(agg_data[features2aggdata], how='left', on='УНОМ')

    
    events2preds = extract_datetime_features(events2preds, "Дата создания во внешней системе")
    events2preds = add_cyclic_features(events2preds, 'month', 12)
    events2preds = add_cyclic_features(events2preds, 'hour', 24)
    events2preds = add_cyclic_features(events2preds, 'quarter', 4)
    events2preds = add_cyclic_features(events2preds, 'day', 31)
    events2preds = add_cyclic_features(events2preds, 'dayofweek', 31)
    events2preds = add_cyclic_features(events2preds, 'weekofyear', 31)
    events2preds.drop("Дата создания во внешней системе", axis=1, inplace=True)
    

    numerical_features = [col for col in events2preds.columns if col not in categorical_features]
    
    # Удаление строк с NaN в числовых признаках
    events2preds[numerical_features] = events2preds[numerical_features].fillna(events2preds[numerical_features].mean())
    
    
    
    for col in categorical_features:
        events2preds[col] = events2preds[col].astype(str)
        
    preds = model.predict_proba(events2preds[features])
    
    return preds[0].tolist()


result_dict = get_predict_for_one(model, 28413, datetime.datetime(2023, 12, 1))
result_dict

/tmp/ipykernel_16530/554794553.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(
/tmp/ipykernel_16530/1458521846.py:104: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odpu = pd.read_sql(
/tmp/ipykernel_16530/1255110072.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  events = pd.read_sql(
/tmp/ipykernel_16530/2569175987.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=Non

[0.22195504037612387,
 0.014008981192959912,
 0.0016114158179154632,
 0.7338227387024742,
 0.028601823910526573]

In [34]:
ftrs2odpu = [
    'mean_volume1forhour',
    'std_volume1forhour',
    'mean_volume2forhour',
    'std_volume2forhour',
    'mean_q2forhour',
    'std_q2forhour',
    'min_volume1forhour',
    'max_volume1forhour',
    'median_volume1forhour',
    'min_volume2forhour',
    'max_volume2forhour',
    'median_volume2forhour',
    'min_q2forhour',
    'max_q2forhour',
    'median_q2forhour',
    'Потребители',
    'Группа',
    'Центральное отопление(контур)',
    'Ошибки'
]


def add_opdu_features(odpu, row):
    local_odpu = odpu[odpu['UNOM'] == row['УНОМ']]
    curr_time = row['Дата создания во внешней системе']
    local_odpu = local_odpu[local_odpu['Месяц/Год'] < curr_time]
    if len(local_odpu) > 14:
        local_odpu = local_odpu.iloc[-14:]

    for feature in ['volume1forhour', 'volume2forhour', 'q2forhour']:
        row[f'mean_{feature}'] = local_odpu[feature].mean()
        row[f'std_{feature}'] = local_odpu[feature].std()
        row[f'min_{feature}'] = local_odpu[feature].min()
        row[f'max_{feature}'] = local_odpu[feature].max()
        row[f'median_{feature}'] = local_odpu[feature].median()
    if len(local_odpu) > 0:
        row['Потребители'] = local_odpu['Потребители'].iloc[-1]
        row['Группа'] = local_odpu['Группа'].iloc[-1]
        row['Центральное отопление(контур)'] = local_odpu['Центральное отопление(контур)'].iloc[-1]
        row['Ошибки'] = local_odpu['Ошибки'].iloc[-1]
    else:
        row['Потребители'] = None
        row['Группа'] = None
        row['Центральное отопление(контур)'] = None
        row['Ошибки'] = None
    return row[ftrs2odpu]

    
def get_odpu(date:datetime.datetime, conn) -> pd.DataFrame: # возвращает таблицу odpu, ОБРЕЗАННУЮ ПО ДАТЕ 
    odpu = pd.read_sql(
       f"""
        SELECT
            *
        FROM 
            heating_data
        WHERE
            to_date(month_year, 'DD-MM-YYYY') BETWEEN to_date('{(date - pd.Timedelta(days=14)).strftime("%d-%m-%Y")}', 'DD-MM-YYYY')
            AND to_date('{date.strftime("%d-%m-%Y")}', 'DD-MM-YYYY')
        ;
        """, conn
    )
    odpu.rename(columns={
        'unom': 'UNOM',
        'id_uu': 'ID УУ',
        'id_tu': 'ID ТУ',
        'district': 'Округ',
        'area': 'Район',
        'consumers': 'Потребители',
        'group_type': 'Группа',
        'address': 'Адрес',
        'central_heating_contour': 'Центральное отопление(контур)',
        'meter_brand': 'Марка счетчика ',
        'meter_serial_number': 'Серия/Номер счетчика',
        'date': 'Дата',
        'month_year': 'Месяц/Год',
        'unit': 'Unit',
        'volume_supplied': 'Объём поданого теплоносителя в систему ЦО',
        'volume_returned': 'Объём обратного теплоносителя из системы ЦО',
        'difference_supply_return_mix': 'Разница между подачей и обраткой(Подмес)',
        'difference_supply_return_leak': 'Разница между подачей и обраткой(Утечка)',
        'temperature_supply': 'Температура подачи',
        'temperature_return': 'Температура обратки',
        'meter_operating_hours': 'Наработка часов счётчика',
        'heat_energy_consumption': 'Расход тепловой энергии',
        'errors': 'Ошибки'
    }, inplace=True)
    odpu['Месяц/Год'] = pd.to_datetime(odpu['Месяц/Год'], format='%d-%m-%Y')
    opdu = odpu[(odpu['Месяц/Год'] <= date) &
                (odpu['Месяц/Год'] >= date - pd.Timedelta(days=14))] 
    
    odpu.sort_values(by='Месяц/Год', inplace=True, ignore_index=True)
    odpu['Объём поданого теплоносителя в систему ЦО'] = odpu['Объём поданого теплоносителя в систему ЦО'].astype(float)
    odpu['volume1forhour'] = odpu['Объём поданого теплоносителя в систему ЦО'] / (odpu['Наработка часов счётчика'])
    odpu['volume2forhour'] = odpu['Объём обратного теплоносителя из системы ЦО'] / (odpu['Наработка часов счётчика'])
    odpu['q2forhour'] = odpu['Расход тепловой энергии'].astype(float) / (odpu['Наработка часов счётчика'])
    

    
    return odpu


def get_odpu_one(unom: int, date: datetime.datetime, conn):
    odpu = pd.read_sql(
       f"""
        SELECT
            *
        FROM 
            heating_data
        WHERE
            heating_data.unom = {unom}
            AND to_date(month_year, 'DD-MM-YYYY') BETWEEN to_date('{(date - pd.Timedelta(days=14)).strftime("%d-%m-%Y")}', 'DD-MM-YYYY')
            AND to_date('{date.strftime("%d-%m-%Y")}', 'DD-MM-YYYY')
        ;
        """, conn
    )
    odpu.rename(columns={
        'unom': 'UNOM',
        'id_uu': 'ID УУ',
        'id_tu': 'ID ТУ',
        'district': 'Округ',
        'area': 'Район',
        'consumers': 'Потребители',
        'group_type': 'Группа',
        'address': 'Адрес',
        'central_heating_contour': 'Центральное отопление(контур)',
        'meter_brand': 'Марка счетчика ',
        'meter_serial_number': 'Серия/Номер счетчика',
        'date': 'Дата',
        'month_year': 'Месяц/Год',
        'unit': 'Unit',
        'volume_supplied': 'Объём поданого теплоносителя в систему ЦО',
        'volume_returned': 'Объём обратного теплоносителя из системы ЦО',
        'difference_supply_return_mix': 'Разница между подачей и обраткой(Подмес)',
        'difference_supply_return_leak': 'Разница между подачей и обраткой(Утечка)',
        'temperature_supply': 'Температура подачи',
        'temperature_return': 'Температура обратки',
        'meter_operating_hours': 'Наработка часов счётчика',
        'heat_energy_consumption': 'Расход тепловой энергии',
        'errors': 'Ошибки'
    }, inplace=True)
    odpu['Месяц/Год'] = pd.to_datetime(odpu['Месяц/Год'], format='%d-%m-%Y')
    opdu = odpu[(odpu['Месяц/Год'] <= date) &
                (odpu['Месяц/Год'] >= date - pd.Timedelta(days=14))] 
    
    odpu.sort_values(by='Месяц/Год', inplace=True, ignore_index=True)
    odpu['Объём поданого теплоносителя в систему ЦО'] = odpu['Объём поданого теплоносителя в систему ЦО'].astype(float)
    odpu['volume1forhour'] = odpu['Объём поданого теплоносителя в систему ЦО'] / (odpu['Наработка часов счётчика'])
    odpu['volume2forhour'] = odpu['Объём обратного теплоносителя из системы ЦО'] / (odpu['Наработка часов счётчика'])
    odpu['q2forhour'] = odpu['Расход тепловой энергии'].astype(float) / (odpu['Наработка часов счётчика'])

    return odpu

In [10]:
get_odpu(datetime.datetime(2023, 12, 30), conn)

/tmp/ipykernel_15196/754678569.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odpu = pd.read_sql(


,UNOM,ID УУ,ID ТУ,Округ,Район,Потребители,Группа,Адрес,Центральное отопление(контур),Марка счетчика,...,Разница между подачей и обраткой(Подмес),Разница между подачей и обраткой(Утечка),Температура подачи,Температура обратки,Наработка часов счётчика,Расход тепловой энергии,Ошибки,volume1forhour,volume2forhour,q2forhour
0,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.00000,0.97266,85.00236,52.05824,24.0,5.92920,,7.486410,7.445882,0.247050
1,2403863,158367,158369,ВАО,Косино-Ухтомский,ГБОУ Школа № 2031,Бюджет,"ул. Медведева, д. 14",ЦО1,КМ-5-2,...,0.00000,2.07568,104.64685,79.31510,24.0,2.37151,,3.881633,3.795146,0.098813
2,2504695,10874,15649,ВАО,Восточный,"ГБУ ДО МКСШОР ""Восток""",Бюджет,"Западная ул., д. 16Б, стр. 5",Вент1,КМ-5-2,...,0.00000,0.00000,18.68100,19.57984,24.0,0.00000,,0.000000,0.000000,0.000000
3,240017,32053,32519,ВАО,Косино-Ухтомский,"ГБУ ""ЖИЛИЩНИК РАЙОНА КОСИНО-УХТОМСКИЙ""",МКД,"ул. Руднёвка, д. 2",ЦО1,КМ-5-2,...,0.89062,0.00000,62.20285,50.98985,24.0,7.30750,,27.124675,27.161784,0.304479
4,2114432,159302,159304,ВАО,Гольяново,ГБОУ Школа № 319,Бюджет,"Амурская ул., д. 6",ЦО1,КМ-5-2,...,3.19531,0.00000,99.24621,68.64416,24.0,4.80005,U,6.515706,6.648845,0.200002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,2114457,146999,147000,ВАО,Гольяново,ГБОУ Школа № 1078,Бюджет,"Алтайская ул., д. 2А",ЦО1,МКТС,...,0.00000,1.03906,91.50000,51.63000,24.0,2.17725,,2.272135,2.228841,0.090719
2029,2115707,147135,147138,ВАО,Восточное Измайлово,ГБОУ Школа № 1811,Бюджет,"Нижняя Первомайская ул., д. 51",ЦО1,ЭСКО МТР-06,...,1.11942,0.00000,92.70703,69.60156,24.0,7.01611,,12.621657,12.668299,0.292338
2030,8672,7203,20759,ВАО,Измайлово,"ГБУК г. Москвы ""ОКЦ ВАО""",Бюджет,"Измайловский пр., д. 18, к. 2 (Ввод 2)",ЦО1,ТеРосс-ТМ,...,0.00000,0.26611,82.61778,60.54066,24.0,0.72235,,1.361247,1.350159,0.030098
2031,2109233,144385,144386,ВАО,Богородское,ГБОУ Школа № 1360,Бюджет,"Краснобогатырская ул., д. 25А",ЦО1,МКТС,...,0.96875,0.00000,91.44000,59.53000,24.0,3.04834,,3.972005,4.012370,0.127014


In [16]:
get_odpu_one(82024, datetime.datetime(2023, 12, 30), conn)

/tmp/ipykernel_15196/2443423781.py:104: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odpu = pd.read_sql(


,UNOM,ID УУ,ID ТУ,Округ,Район,Потребители,Группа,Адрес,Центральное отопление(контур),Марка счетчика,...,Разница между подачей и обраткой(Подмес),Разница между подачей и обраткой(Утечка),Температура подачи,Температура обратки,Наработка часов счётчика,Расход тепловой энергии,Ошибки,volume1forhour,volume2forhour,q2forhour
0,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.97266,85.00236,52.05824,24.0,5.92920,,7.486410,7.445882,0.247050
1,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.88086,76.89833,47.33347,24.0,4.75610,,6.694580,6.657878,0.198171
2,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.89453,70.79049,45.31535,24.0,4.26093,,6.960856,6.923584,0.177539
3,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.87891,67.39267,43.83624,24.0,3.85962,,6.818929,6.782308,0.160818
4,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.86524,65.75852,42.87904,24.0,3.60053,,6.548828,6.512777,0.150022
5,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.92969,67.67542,46.06925,24.0,4.26135,,8.206950,8.168213,0.177556
6,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.98047,67.52190,47.88139,24.0,4.61701,,9.781006,9.740153,0.192375
7,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.83008,68.66834,46.46072,24.0,4.26172,,7.984945,7.950358,0.177572
8,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.94336,68.41111,47.69375,24.0,4.52759,,9.093262,9.053955,0.188650
9,82024,5005,16213,ВАО,Новокосино,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,"Новокосинская ул., д. 12, к. 5",ЦО1,АТ-Т-2,...,0.0,0.92578,70.54332,48.38593,24.0,4.72229,,8.868001,8.829427,0.196762


In [31]:
def get_agg_data(conn) -> pd.DataFrame:
    df = pd.read_sql_query(
        """ 
        SELECT *
        FROM buildings
        """, conn
    )
    new_column_names = {
        'external_system_address': 'Адрес из сторонней системы',
        'external_system_id': 'Идентификатор из сторонней системы',
        'bti_address': 'Адрес по БТИ',
        'unom': 'unom',
        'district': 'Округ',
        'area': 'Район',
        'project_series': 'Серии проектов',
        'number_of_floors': 'Количество этажей',
        'number_of_entrances': 'Количество подъездов',
        'number_of_apartments': 'Количество квартир',
        'total_area': 'Общая площадь',
        'total_residential_area': 'Общая площадь жилых помещений',
        'total_non_residential_area': 'Общая площадь нежилых помещений',
        'wear_and_tear_bti': 'Износ объекта (по БТИ)',
        'wall_materials': 'Материалы стен',
        'emergency_status': 'Признак аварийности здания',
        'number_of_passenger_elevators': 'Количество пассажирских лифтов',
        'number_of_freight_elevators': 'Количество грузопассажирских лифтов',
        'roof_cleaning_priority': 'Очередность уборки кровли',
        'roof_materials': 'Материалы кровли по БТИ',
        'housing_fund_types': 'Типы жилищного фонда',
        'mkd_statuses': 'Статусы МКД',
        'consumers': 'Потребители',
        'group_type': 'Группа',
        'central_heating': 'Центральное отопление(контур)',
        'meter_brand': 'Марка счетчика ',
        'meter_serial_number': 'Серия/Номер счетчика',
        'id_uu': 'ID УУ',
        'full_address': 'Полный адрес',
        'ods_number': '№ ОДС',
        'ods_address': 'Адрес ОДС',
        'ctp': 'ЦТП',
        'serial_number': '№ п/п',
        'city': 'Город',
        'administrative_district': 'Административный округ',
        'municipal_district': 'Муниципальный округ',
        'locality': 'Населенный пункт',
        'street': 'Улица',
        'house_number_type': 'Тип номера дом',
        'house_number': 'Номер дома',
        'building_number': 'Номер корпуса',
        'structure_number_type': 'Тип номера строения/сооружения',
        'structure_number': 'Номер строения',
        'unad': 'UNAD',
        'material': 'Материал',
        'purpose': 'Назначение',
        'class': 'Класс',
        'type': 'Тип',
        'sign': 'Признак',
        'global_id': 'global_id',
        'obj_type': 'OBJ_TYPE',
        'address_x': 'ADDRESS_x',
        'municipal_district_1': 'Муниципальный округ',
        'planning_element_name': 'Наименование элемента планировочной структуры или улично-дорожной сети',
        'house_ownership_number_type': 'Тип номера дома, владения, участка',
        'intra_city_area': 'Внутригородская территория',
        'adm_area': 'ADM_AREA',
        'district_1': 'DISTRICT',
        'nreg': 'NREG',
        'dreg': 'DREG',
        'n_fias': 'N_FIAS',
        'd_fias': 'D_FIAS',
        'kad_n': 'KAD_N',
        'kad_zu': 'KAD_ZU',
        'kladr': 'KLADR',
        'tdoc': 'TDOC',
        'ndoc': 'NDOC',
        'ddoc': 'DDOC',
        'adr_type': 'ADR_TYPE',
        'vid': 'VID',
        'sostad': 'SOSTAD',
        'status': 'STATUS',
        'geo_data': 'geoData',
        'geo_data_center': 'geodata_center',
        'id_ods': 'ID ODS',
        'phone_number': 'PHONE_NUMBER'
    }
    
    # Переименование столбцов
    df.rename(columns=new_column_names, inplace=True)
    return df

In [13]:
get_agg_data(conn)

/tmp/ipykernel_16530/2218722158.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


,unom,ЦТП,Адрес из сторонней системы,Идентификатор из сторонней системы,Адрес по БТИ,Округ,Район,Серии проектов,Количество этажей,Количество подъездов,Количество квартир,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Износ объекта (по БТИ),Материалы стен,Признак аварийности здания,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Очередность уборки кровли,Материалы кровли по БТИ,Типы жилищного фонда,Статусы МКД,Потребители,Группа,Центральное отопление(контур),Марка счетчика,Серия/Номер счетчика,ID УУ,Полный адрес,№ ОДС,Адрес ОДС,№ п/п,Город,Административный округ,Муниципальный округ,Населенный пункт,Улица,Тип номера дом,Номер дома,Номер корпуса,Тип номера строения/сооружения,Номер строения,UNAD,Материал,Назначение,Класс,Тип,Признак,global_id,OBJ_TYPE,ADDRESS_x,Наименование элемента планировочной структуры или улично-дорожной сети,"Тип номера дома, владения, участка",Внутригородская территория,ADM_AREA,DISTRICT,NREG,DREG,N_FIAS,D_FIAS,KAD_N,KAD_ZU,KLADR,TDOC,NDOC,DDOC,ADR_TYPE,VID,SOSTAD,STATUS,geoData,geodata_center,ID ODS,PHONE_NUMBER
0,28413,04-07-0615/016,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Ив...",1537264.0,"город Москва, Челябинская улица, дом 22, корпус 1",Восточный административный округ,муниципальный округ Ивановское,И-209А,14.0,1.0,98.0,4246.8,4246.8,0,,легкобетонные блоки,22728486.0,2.0,0.0,,мягкая-совмещенная с рубероидным покрытием,42875644.0,58761330.0,"ГБУ ""ЖИЛИЩНИК ИВАНОВСКОГО РАЙОНА""",МКД,ЦО1,АТ-Т-2,603024.0,7.854877e+08,"город Москва, Челябинская улица, дом 22, корпус 1",ОДС №3-Ивановское,"город Москва, улица Чечулина, дом 26",82457.0,город Москва,Восточный административный округ,муниципальный округ Ивановское,,Челябинская улица,дом,22,1,,,1.0,легкобетонные блоки,многоквартирный дом,жилые,Здание,,647310308.0,Здание,"Российская Федерация, город Москва, внутригоро...",Челябинская улица,дом,внутригородская территория муниципальный округ...,Восточный административный округ,муниципальный округ Ивановское,3301397,03.09.2003,45694AB2-CA73-4182-B61C-7126B04C2276,27.02.2012,KAD_N:77:03:0005024:1099\n\n,,77000000000308000,Распоряжение префектуры АО города Москвы,782-В-РП,31.07.2003,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"[[[37.83223992, 55.777896607], [37.832458926, ...","[37.832467254, 55.777764326]",143495047.0,8 (495) 539-53-53
1,18744,04-05-0604/024-10,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Из...",1524733.0,"город Москва, 9-я Парковая улица, дом 50, корп...",Восточный административный округ,муниципальный округ Измайлово,нет данных,5.0,3.0,60.0,2569.0,2569.0,0,,из железобетонных сегментов,22728486.0,0.0,0.0,22289163.0,мягкая-совмещенная с рубероидным покрытием,42875644.0,58761330.0,"ГБУ ""ЖИЛИЩНИК РАЙОНА ИЗМАЙЛОВО""",МКД,ЦО1,ВЗЛЕТ ТСРВ,1900203.0,7.854786e+08,"город Москва, 9-я Парковая улица, дом 50, корп...",ОДС №27-Измайлово,"город Москва, 7-я Парковая улица, дом 15, корп...",69005.0,город Москва,Восточный административный округ,муниципальный округ Измайлово,,9-я Парковая улица,дом,50,2,,,1.0,из железобетонных сегментов,многоквартирный дом,жилые,Здание,,646457116.0,Здание,"Российская Федерация, город Москва, внутригоро...",9-я Парковая улица,дом,внутригородская территория муниципальный округ...,Восточный административный округ,муниципальный округ Измайлово,3302446,06.01.2004,354ED872-79FE-4863-93E6-022D72D0D6A1,27.02.2012,KAD_N:77:03:0005012:1051\n\n,,77000000000222200,Распоряжение префектуры АО города Москвы,1393-В-РП,24.12.2003,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"[[[37.800624269, 55.800754197], [37.800721977,...","[37.800636977, 55.800520274]",143495010.0,8 (495) 539-53-53
2,18797,04-05-0604/124,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Из...",1514520.0,"город Москва, 11-я Парковая улица, дом 31",Восточный административный округ,муниципальный округ Измайлово,индивидуальный проект,5.0,3.0,50.0,3619.2,3591.3,2024-09-27 00:00:00,,кирпичные,22728486.0,0.0,0.0,22289162.0,стальная,42875644.0,58761330.0,"ГБУ ""ЖИЛИЩНИК РАЙОНА

In [32]:
def get_agg_data_one(unom: int, conn) -> pd.DataFrame:
    df = pd.read_sql_query(
        f""" 
        SELECT *
        FROM buildings
        WHERE buildings.unom = {unom}
        """, conn
    )
    new_column_names = {
        'external_system_address': 'Адрес из сторонней системы',
        'external_system_id': 'Идентификатор из сторонней системы',
        'bti_address': 'Адрес по БТИ',
        'unom': 'unom',
        'district': 'Округ',
        'area': 'Район',
        'project_series': 'Серии проектов',
        'number_of_floors': 'Количество этажей',
        'number_of_entrances': 'Количество подъездов',
        'number_of_apartments': 'Количество квартир',
        'total_area': 'Общая площадь',
        'total_residential_area': 'Общая площадь жилых помещений',
        'total_non_residential_area': 'Общая площадь нежилых помещений',
        'wear_and_tear_bti': 'Износ объекта (по БТИ)',
        'wall_materials': 'Материалы стен',
        'emergency_status': 'Признак аварийности здания',
        'number_of_passenger_elevators': 'Количество пассажирских лифтов',
        'number_of_freight_elevators': 'Количество грузопассажирских лифтов',
        'roof_cleaning_priority': 'Очередность уборки кровли',
        'roof_materials': 'Материалы кровли по БТИ',
        'housing_fund_types': 'Типы жилищного фонда',
        'mkd_statuses': 'Статусы МКД',
        'consumers': 'Потребители',
        'group_type': 'Группа',
        'central_heating': 'Центральное отопление(контур)',
        'meter_brand': 'Марка счетчика ',
        'meter_serial_number': 'Серия/Номер счетчика',
        'id_uu': 'ID УУ',
        'full_address': 'Полный адрес',
        'ods_number': '№ ОДС',
        'ods_address': 'Адрес ОДС',
        'ctp': 'ЦТП',
        'serial_number': '№ п/п',
        'city': 'Город',
        'administrative_district': 'Административный округ',
        'municipal_district': 'Муниципальный округ',
        'locality': 'Населенный пункт',
        'street': 'Улица',
        'house_number_type': 'Тип номера дом',
        'house_number': 'Номер дома',
        'building_number': 'Номер корпуса',
        'structure_number_type': 'Тип номера строения/сооружения',
        'structure_number': 'Номер строения',
        'unad': 'UNAD',
        'material': 'Материал',
        'purpose': 'Назначение',
        'class': 'Класс',
        'type': 'Тип',
        'sign': 'Признак',
        'global_id': 'global_id',
        'obj_type': 'OBJ_TYPE',
        'address_x': 'ADDRESS_x',
        'municipal_district_1': 'Муниципальный округ',
        'planning_element_name': 'Наименование элемента планировочной структуры или улично-дорожной сети',
        'house_ownership_number_type': 'Тип номера дома, владения, участка',
        'intra_city_area': 'Внутригородская территория',
        'adm_area': 'ADM_AREA',
        'district_1': 'DISTRICT',
        'nreg': 'NREG',
        'dreg': 'DREG',
        'n_fias': 'N_FIAS',
        'd_fias': 'D_FIAS',
        'kad_n': 'KAD_N',
        'kad_zu': 'KAD_ZU',
        'kladr': 'KLADR',
        'tdoc': 'TDOC',
        'ndoc': 'NDOC',
        'ddoc': 'DDOC',
        'adr_type': 'ADR_TYPE',
        'vid': 'VID',
        'sostad': 'SOSTAD',
        'status': 'STATUS',
        'geo_data': 'geoData',
        'geo_data_center': 'geodata_center',
        'id_ods': 'ID ODS',
        'phone_number': 'PHONE_NUMBER'
    }
    
    # Переименование столбцов
    df.rename(columns=new_column_names, inplace=True)
    return df

In [15]:

get_agg_data_one(82024, conn)

/tmp/ipykernel_16530/1097622698.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


,unom,ЦТП,Адрес из сторонней системы,Идентификатор из сторонней системы,Адрес по БТИ,Округ,Район,Серии проектов,Количество этажей,Количество подъездов,Количество квартир,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Износ объекта (по БТИ),Материалы стен,Признак аварийности здания,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Очередность уборки кровли,Материалы кровли по БТИ,Типы жилищного фонда,Статусы МКД,Потребители,Группа,Центральное отопление(контур),Марка счетчика,Серия/Номер счетчика,ID УУ,Полный адрес,№ ОДС,Адрес ОДС,№ п/п,Город,Административный округ,Муниципальный округ,Населенный пункт,Улица,Тип номера дом,Номер дома,Номер корпуса,Тип номера строения/сооружения,Номер строения,UNAD,Материал,Назначение,Класс,Тип,Признак,global_id,OBJ_TYPE,ADDRESS_x,Наименование элемента планировочной структуры или улично-дорожной сети,"Тип номера дома, владения, участка",Внутригородская территория,ADM_AREA,DISTRICT,NREG,DREG,N_FIAS,D_FIAS,KAD_N,KAD_ZU,KLADR,TDOC,NDOC,DDOC,ADR_TYPE,VID,SOSTAD,STATUS,geoData,geodata_center,ID ODS,PHONE_NUMBER
0,82024,04-09-302,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Но...",1539626.0,"город Москва, Новокосинская улица, дом 12, кор...",Восточный административный округ,муниципальный округ Новокосино,П-3/16,17.0,2.0,136.0,8694.7,8677.4,2024-03-17 00:00:00,,панельные,22728486.0,2.0,2.0,,мягкая-совмещенная с рубероидным покрытием,42875644.0,58761330.0,"ГБУ ""ЖИЛИЩНИК РАЙОНА НОВОКОСИНО""",МКД,ЦО1,АТ-Т-2,614289.0,5005.0,"город Москва, Новокосинская улица, дом 12, кор...",,,64067.0,город Москва,Восточный административный округ,муниципальный округ Новокосино,,Новокосинская улица,дом,12,5,,,2.0,панельные,многоквартирный дом,жилые,Здание,,648983224.0,Здание,"Российская Федерация, город Москва, внутригоро...",Новокосинская улица,дом,внутригородская территория муниципальный округ...,Восточный административный округ,муниципальный округ Новокосино,3200169,25.02.2000,B2E08CE7-3F13-4301-8A4C-66291AC31F12,27.02.2012,KAD_N:77:03:0009004:1118\n\n,KAD_ZU:77:03:0009004:16195\n\n,77000000000042200,Решение исполкома райсовета,888,14.03.1990,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"[[[37.857500194, 55.736633236], [37.85771732, ...","[37.857673062, 55.736430968]",None,


In [16]:
feature2events = [
    'Давление не в норме_count', 
    'T < min_count', 
    'T > max_count', 
    'Утечка_count', 
    'month1_count', 
    'month2_count', 
    'month3_count', 
    'month4_count', 
    'month10_count', 
    'month11_count', 
    'month12_count'
]

new_event_names = [
    'Давление не в норме',
    'T < min',
    'T > max',
    'Утечка'
]

def collect_events(row, events):
    local_events = events[events['УНОМ'] == row['УНОМ']]
    curr_time = row['Дата создания во внешней системе']
    local_events = local_events[local_events['Дата создания во внешней системе'] > curr_time]
    grpb = local_events.groupby('Наименование').count()['УНОМ']
    for event_type in new_event_names:
        if event_type in grpb:
            row[f'{event_type}_count'] = grpb[event_type]
        else:
            row[f'{event_type}_count'] = 0

    grpb = local_events.groupby('month').count()['Наименование']
    for month_num in [1,2,3,4,10,11,12]:
        if month_num in grpb:
            row[f"month{month_num}_count"] = grpb[month_num]
        else:
            row[f"month{month_num}_count"] = 0
    
    return row[feature2events]

In [17]:
num2month = {
    10: "october",
    11: "november",
    12: "december",
    1: "january",
    2: "february",
    3: "march",
    4: "april",
    5: "may",
    6: "june",
    7: "july",
    8: "august",
    9: "september"
}

def collect_weather(row, weather):
    mon = num2month[row.month]
    day = "{:02d}".format(row.day)
    row['weather1'], row['weather2'] = weather[mon][day]
    return row[['weather1', 'weather2']]

In [18]:
def get_events(date, conn) -> pd.DataFrame:
    events = pd.read_sql(
        f"""
        SELECT *
        FROM events
        WHERE 
            events.creation_date <= '{date.strftime("%Y-%m-%d")}'
            AND events.creation_date >= '2023-10-01'
        """, conn
    )
    events.rename(columns={
        'name': 'Наименование', 
        'source': 'Источник', 
        'creation_date': 'Дата создания во внешней системе', 
        'closure_date': 'Дата закрытия', 
        'district': 'Округ',
        'unom': 'УНОМ',
       'address': 'Адрес', 
        'event_completion_date': 'Дата и время завершения события'
    }, inplace=True)
    
    eventNames2labels = {
        "P1 <= 0": "Давление не в норме",
        "P2 <= 0": "Давление не в норме",
        "T1 > max": "T > max",
        "T1 < min": "T < min",
        "Недостаточная температура подачи в центральном отоплении (Недотоп)": "T < min",
        "Превышение температуры подачи в центральном отоплении (Перетоп)": "T > max",
        "Утечка теплоносителя": "Утечка",
        "Течь в системе отопления": "Утечка",
        "Температура в квартире ниже нормативной": "T < min",
        "Отсутствие отопления в доме": "T < min",
        "Сильная течь в системе отопления": "Утечка",
        "Температура в помещении общего пользования ниже нормативной": "T < min",
        "Аварийная протечка труб в подъезде": "Утечка",
        "Протечка труб в подъезде": "Утечка",
        "Температура в помещении общего пользования ниже нормативной": "T < min",
        "Отсутствие отопления в доме": "T < min",
        "Температура в квартире ниже нормативной": "T < min",
        "Течь в системе отопления": "Утечка",
        "Сильная течь в системе отопления": "Утечка",
    
    }
    
    event_names = list(eventNames2labels.keys())
    events = events[events['Наименование'].isin(event_names)]
    events['Наименование'] = events['Наименование'].apply(lambda x: eventNames2labels[x])
    events['Дата создания во внешней системе'] = pd.to_datetime(events['Дата создания во внешней системе'])
    events.sort_values(by='Дата создания во внешней системе', inplace=True, ignore_index=True)
    events['month'] = events['Дата создания во внешней системе'].apply(lambda x: x.month)
    events['day'] = events['Дата создания во внешней системе'].apply(lambda x: x.day)
    events = events[['Наименование', 'Источник', 'Дата создания во внешней системе', 'УНОМ', 'month', 'day']]

    return events


def get_events_one(unom: int, date: datetime.datetime, conn) -> pd.DataFrame:
    events = pd.read_sql(
        f"""
        SELECT *
        FROM events
        WHERE 
            events.creation_date <= '{date.strftime("%Y-%m-%d")}'
            AND events.creation_date >= '2023-10-01'
            AND unom = {unom}
        """, conn
    )
    events.rename(columns={
        'name': 'Наименование', 
        'source': 'Источник', 
        'creation_date': 'Дата создания во внешней системе', 
        'closure_date': 'Дата закрытия', 
        'district': 'Округ',
        'unom': 'УНОМ',
       'address': 'Адрес', 
        'event_completion_date': 'Дата и время завершения события'
    }, inplace=True)
    
    eventNames2labels = {
        "P1 <= 0": "Давление не в норме",
        "P2 <= 0": "Давление не в норме",
        "T1 > max": "T > max",
        "T1 < min": "T < min",
        "Недостаточная температура подачи в центральном отоплении (Недотоп)": "T < min",
        "Превышение температуры подачи в центральном отоплении (Перетоп)": "T > max",
        "Утечка теплоносителя": "Утечка",
        "Течь в системе отопления": "Утечка",
        "Температура в квартире ниже нормативной": "T < min",
        "Отсутствие отопления в доме": "T < min",
        "Сильная течь в системе отопления": "Утечка",
        "Температура в помещении общего пользования ниже нормативной": "T < min",
        "Аварийная протечка труб в подъезде": "Утечка",
        "Протечка труб в подъезде": "Утечка",
        "Температура в помещении общего пользования ниже нормативной": "T < min",
        "Отсутствие отопления в доме": "T < min",
        "Температура в квартире ниже нормативной": "T < min",
        "Течь в системе отопления": "Утечка",
        "Сильная течь в системе отопления": "Утечка",
    
    }
    
    event_names = list(eventNames2labels.keys())
    events = events[events['Наименование'].isin(event_names)]
    events['Наименование'] = events['Наименование'].apply(lambda x: eventNames2labels[x])
    events['Дата создания во внешней системе'] = pd.to_datetime(events['Дата создания во внешней системе'])
    events.sort_values(by='Дата создания во внешней системе', inplace=True, ignore_index=True)
    events['month'] = events['Дата создания во внешней системе'].apply(lambda x: x.month)
    events['day'] = events['Дата создания во внешней системе'].apply(lambda x: x.day)
    events = events[['Наименование', 'Источник', 'Дата создания во внешней системе', 'УНОМ', 'month', 'day']]

    return events

In [39]:
get_events(datetime.datetime(2023, 12, 1), conn)

/tmp/ipykernel_15196/269833220.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  events = pd.read_sql(


,Наименование,Источник,Дата создания во внешней системе,УНОМ,month,day
0,T < min,EDC,2023-10-01 05:41:02,15109,10,1
1,T < min,EDC,2023-10-01 05:41:02,15109,10,1
2,T < min,EDC,2023-10-01 05:41:14,15109,10,1
3,T < min,EDC,2023-10-01 05:41:14,15109,10,1
4,T < min,EDC,2023-10-01 07:13:35,24224,10,1
...,...,...,...,...,...,...
122823,T < min,EDC,2023-11-30 23:41:44,24412,11,30
122824,T < min,ASUPR,2023-11-30 23:53:28,18866,11,30
122825,T < min,ASUPR,2023-11-30 23:53:28,18866,11,30
122826,T < min,EDC,2023-11-30 23:59:55,1206,11,30


In [41]:
get_events_one(15109, datetime.datetime(2023, 12, 1), conn)

/tmp/ipykernel_15196/1255110072.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  events = pd.read_sql(


,Наименование,Источник,Дата создания во внешней системе,УНОМ,month,day
0,T < min,EDC,2023-10-01 05:41:02,15109,10,1
1,T < min,EDC,2023-10-01 05:41:02,15109,10,1
2,T < min,EDC,2023-10-01 05:41:14,15109,10,1
3,T < min,EDC,2023-10-01 05:41:14,15109,10,1
4,T < min,EDC,2023-10-08 09:39:05,15109,10,8
...,...,...,...,...,...,...
155,T < min,EDC,2023-11-20 09:20:13,15109,11,20
156,T < min,NG,2023-11-21 19:01:43,15109,11,21
157,T < min,NG,2023-11-21 19:01:43,15109,11,21
158,Утечка,EDC,2023-11-22 12:42:19,15109,11,22


In [19]:
def extract_datetime_features(df, date_col):
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day
    df['hour'] = df[date_col].dt.hour
    df['dayofweek'] = df[date_col].dt.dayofweek
    df['weekofyear'] = df[date_col].dt.isocalendar().week
    df['quarter'] = df[date_col].dt.quarter
    df['is_month_start'] = df[date_col].dt.is_month_start
    df['is_month_end'] = df[date_col].dt.is_month_end
    return df
    
    
def add_cyclic_features(df, col, max_val):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    return df